Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Author: Sebastian Raschka

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 8.30.0

torch: 2.6.0+cu126



# Network in Network CIFAR-10 Classifier

based on 

- Lin, Min, Qiang Chen, and Shuicheng Yan. "Network in network." arXiv preprint arXiv:1312.4400 (2013).

## Imports

In [2]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [3]:
torch.tensor([1]).device

device(type='cpu')

## Model Settings

In [4]:
##########################
### 设置
##########################

# 超参数
RANDOM_SEED = 1
LEARNING_RATE = 0.00005
BATCH_SIZE = 256
NUM_EPOCHS = 100

# 网络架构
NUM_CLASSES = 10

# 其他设置
DEVICE = "cuda:0"
GRAYSCALE = False

In [5]:
##########################
### CIFAR-10 数据集
##########################

# 注意 transforms.ToTensor() 会将输入图像缩放到 0-1 范围内

train_indices = torch.arange(0, 49000)  # 训练集的索引
valid_indices = torch.arange(49000, 50000)  # 验证集的索引

# 加载 CIFAR-10 数据集，train=True 表示训练集，transform=transforms.ToTensor() 将图像转为 Tensor 格式，下载数据集
train_and_valid = datasets.CIFAR10(root='data', 
                                   train=True, 
                                   transform=transforms.ToTensor(),
                                   download=True)

# 从 CIFAR-10 数据集中提取训练集和验证集
train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)

# 加载测试集，train=False 表示测试集，transform=transforms.ToTensor() 将图像转为 Tensor 格式
test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())

#####################################################
### 数据加载器
#####################################################

# 创建训练数据加载器，batch_size 为批次大小，num_workers 设置为 8 表示使用 8 个子进程加载数据，shuffle=True 表示打乱数据
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          shuffle=True)

# 创建验证数据加载器，shuffle=False 表示不打乱验证集数据
valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          shuffle=False)

# 创建测试数据加载器，shuffle=False 表示不打乱测试集数据
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=8,
                         shuffle=False)

#####################################################

# 检查数据集的形状
for images, labels in train_loader:  
    print('训练集图像批次的维度:', images.shape)  # 输出训练集图像的维度
    print('训练集标签批次的维度:', labels.shape)  # 输出训练集标签的维度
    break

for images, labels in test_loader:  
    print('测试集图像批次的维度:', images.shape)  # 输出测试集图像的维度
    print('测试集标签批次的维度:', labels.shape)  # 输出测试集标签的维度
    break
    
for images, labels in valid_loader:  
    print('验证集图像批次的维度:', images.shape)  # 输出验证集图像的维度
    print('验证集标签批次的维度:', labels.shape)  # 输出验证集标签的维度
    break

训练集图像批次的维度: torch.Size([256, 3, 32, 32])
训练集标签批次的维度: torch.Size([256])
测试集图像批次的维度: torch.Size([256, 3, 32, 32])
测试集标签批次的维度: torch.Size([256])
验证集图像批次的维度: torch.Size([256, 3, 32, 32])
验证集标签批次的维度: torch.Size([256])


In [6]:
##########################
### MODEL
##########################


class NiN(nn.Module):
    def __init__(self, num_classes):
        super(NiN, self).__init__()
        self.num_classes = num_classes
        self.classifier = nn.Sequential(
                nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.Conv2d(160,  96, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                nn.Dropout(0.5),

                nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                nn.Dropout(0.5),

                nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.Conv2d(192,  10, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.AvgPool2d(kernel_size=8, stride=1, padding=0),

                )

    def forward(self, x):
        x = self.classifier(x)
        logits = x.view(x.size(0), self.num_classes)
        probas = torch.softmax(logits, dim=1)
        return logits, probas

## Training without Pinned Memory

In [7]:
torch.manual_seed(RANDOM_SEED)

model = NiN(NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [8]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)  # 将特征数据移到指定设备
        targets = targets.to(device)    # 将目标数据移到指定设备

        logits, probas = model(features)  # 获取模型输出的logits和概率
        _, predicted_labels = torch.max(probas, 1)  # 取最大概率的类别作为预测标签
        num_examples += targets.size(0)  # 累加样本数量
        correct_pred += (predicted_labels == targets).sum()  # 累加预测正确的样本数
    return correct_pred.float()/num_examples * 100  # 返回正确预测率百分比
    

start_time = time.time()  # 记录训练开始时间
for epoch in range(NUM_EPOCHS):  # 遍历每个epoch
    
    model.train()  # 设置模型为训练模式
    
    for batch_idx, (features, targets) in enumerate(train_loader):  # 遍历每个batch
    
        ### 准备每个小批量数据
        features = features.to(DEVICE)  # 将特征数据移到指定设备
        targets = targets.to(DEVICE)    # 将目标数据移到指定设备
            
        ### 前向传播和反向传播
        logits, probas = model(features)  # 获取模型输出的logits和概率
        cost = F.cross_entropy(logits, targets)  # 计算交叉熵损失
        optimizer.zero_grad()  # 清除旧的梯度
        
        cost.backward()  # 计算梯度
        
        ### 更新模型参数
        optimizer.step()  # 根据梯度更新模型参数
        
        ### 日志输出
        if not batch_idx % 120:  # 每隔120个batch打印一次日志
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')  # 输出当前epoch和batch的损失值

    # 在计算准确率时不需要构建计算图以节省内存
    with torch.set_grad_enabled(False):
        train_acc = compute_accuracy(model, train_loader, device=DEVICE)  # 计算训练集的准确率
        valid_acc = compute_accuracy(model, valid_loader, device=DEVICE)  # 计算验证集的准确率
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')  # 输出训练集和验证集的准确率
        
    elapsed = (time.time() - start_time)/60  # 计算训练已经进行的时间
    print(f'Time elapsed: {elapsed:.2f} min')  # 输出经过的时间
  
elapsed = (time.time() - start_time)/60  # 计算总的训练时间
print(f'Total Training Time: {elapsed:.2f} min')  # 输出总的训练时间

Epoch: 001/100 | Batch 000/192 | Cost: 2.3041
Epoch: 001/100 | Batch 120/192 | Cost: 2.1722
Epoch: 001/100 Train Acc.: 20.37% | Validation Acc.: 20.40%
Time elapsed: 0.28 min
Epoch: 002/100 | Batch 000/192 | Cost: 2.1120
Epoch: 002/100 | Batch 120/192 | Cost: 2.0647
Epoch: 002/100 Train Acc.: 22.95% | Validation Acc.: 22.60%
Time elapsed: 0.55 min
Epoch: 003/100 | Batch 000/192 | Cost: 1.9668
Epoch: 003/100 | Batch 120/192 | Cost: 1.9961
Epoch: 003/100 Train Acc.: 26.90% | Validation Acc.: 26.40%
Time elapsed: 0.83 min
Epoch: 004/100 | Batch 000/192 | Cost: 1.9166
Epoch: 004/100 | Batch 120/192 | Cost: 1.9316
Epoch: 004/100 Train Acc.: 28.25% | Validation Acc.: 27.80%
Time elapsed: 1.10 min
Epoch: 005/100 | Batch 000/192 | Cost: 1.8351
Epoch: 005/100 | Batch 120/192 | Cost: 1.8305
Epoch: 005/100 Train Acc.: 31.79% | Validation Acc.: 32.80%
Time elapsed: 1.37 min
Epoch: 006/100 | Batch 000/192 | Cost: 1.7605
Epoch: 006/100 | Batch 120/192 | Cost: 1.7812
Epoch: 006/100 Train Acc.: 32.90%

In [9]:
with torch.set_grad_enabled(False): # 在推理过程中禁用梯度计算，节省内存
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 59.22%


In [10]:
%watermark -iv

PIL        : 11.0.0
torchvision: 0.21.0+cu126
torch      : 2.6.0+cu126
numpy      : 2.1.2
matplotlib : 3.10.1
pandas     : 2.2.3

